In [1]:
import gc
import numpy as np
import itertools as it
import math
import random
import string
from collections import namedtuple
from copy import deepcopy

from memory_profiler import memory_usage

random.seed()
x_range = (-100, 100)
y_range = (-100, 100)
z_range = (0, 50)
Point3 = namedtuple('Point3', ['x', 'y', 'z'])

In [7]:
class Graph:
    def __init__(self, n, *, x_range=(-100, 100), y_range=(-100, 100),
                 z_range=(0, 50), symmetrical=True, removed_percent=20, is_height_diff=True):
        self.city_labels = list((string.ascii_uppercase + string.ascii_lowercase + string.digits)[:n])
        self.vertices = {}
        for i in range(n):
            self.vertices[self.city_labels[i]] = Point3(random.uniform(*x_range), random.uniform(*y_range),
                                                        random.uniform(*z_range))

        self.edges = {}
        self.distances = {}
        self.all_edges = list(it.combinations(self.city_labels, 2))
        for city, position_a in self.vertices.items():
            self.edges[city] = []
            self.distances[city] = {}
            for other_city, position_b in self.vertices.items():
                d = self.dist(position_a, position_b)
                if (city == other_city):
                    self.distances[city][other_city] = d
                    continue
                self.edges[city].append(other_city)
                self.distances[city][other_city] = d
        if (removed_percent > 0):
            edges_copy = deepcopy(self.edges)
            items_to_remove = math.ceil(n / 100 * len(self.vertices))
            combs = [*it.combinations(self.edges, 2)]
            choices = random.sample(combs, items_to_remove)
            for choice in choices:
                _from, _to = choice
                self.edges[_from].remove(_to)
                if (symmetrical):
                    self.edges[_to].remove(_from)
                    
                    
    def dist(self, point_a, point_b, height_diff=True):
        d = math.sqrt((point_a.x - point_b.x) ** 2 + (point_a.y - point_b.y) ** 2 + (point_a.z - point_b.z) ** 2)
        if (height_diff):
            if point_a.z < point_b.z:
                d *= 1.1
            elif point_a.z > point_b.z:
                d *= 0.9
        return d

In [9]:
g = Graph(5)
g.edges

{'A': ['B', 'C', 'D', 'E'],
 'B': ['A', 'C', 'D', 'E'],
 'C': ['A', 'B', 'D', 'E'],
 'D': ['A', 'B', 'C'],
 'E': ['A', 'B', 'C']}

In [3]:
class State:
    def __init__(self, pos, previous=[], cost=0, parent=None, visited_edges=None):
        self.pos = pos
        if not visited_edges:
            self.visited_edges = []
        else:
            self.visited_edges = visited_edges.copy()

        if not previous:
            self.previous = [pos]
        else:
            self.previous = previous.copy()
        if len(self.previous) > 1:
            curr = pos
            prev = self.previous[-2]
            self.visited_edges.append((prev, curr))
            self.visited_edges.append((curr, prev))
        self.cost = cost
        self.parent = parent

    def __str__(self):
        return f"<Pos: {self.pos} | Cost: {self.cost:.3f} | Visited {self.previous}>"

    def __repr__(self):
        return self.__str__()

    def get_path(self):
        previous_paths = []
        self.__path(previous_paths)
        return '>'.join(reversed(previous_paths))

    def __path(self, previous):
        previous.append(self.pos)
        if self.parent is not None:
            self.parent.__path(previous)
        return previous

    def __eq__(self, other):
        return self.pos == other.pos and self.previous == other.previous
    
    def __lt__(self, other): #only for the sake of priority conflicts
        return True

    def expand(self, graph):
        children = []
        possible_routes = graph.edges[self.pos]
        for end_edge in possible_routes:
            route_cost = graph.distances[self.pos][end_edge]
            child = State(end_edge, [*self.previous, end_edge], self.cost + route_cost, self, self.visited_edges)
            children.append(child)
        return self._filter_invalid_and_visited(children, graph)

    def _filter_invalid_and_visited(self, potential_children, graph):
        # special case if we have completed a tour and need to return
        children = []
        if (len(self.previous) == len(graph.vertices)):
            initial_city = self.previous[0]
            for child in potential_children:
                if (child.pos == initial_city):
                    children.append(child)
                    # print(f"Returning to {initial_city} from {child}")
        else:
            for child in potential_children:
                if (child.pos not in self.previous):
                    children.append(child)
        # print(f"removed invalid states {[c for c in potential_children if c not in children]}")
        return children

In [4]:
def is_goal_state(state, initial_state, graph):
    if state.pos == initial_state.pos and len(state.previous) == len(graph.vertices) + 1:
        #todo check if all letters are present?
        return True
    return False


def pretty_print(mode, size, fn_to_eval):
    result, t = fn_to_eval()
    print(f"    [{mode} | n={size}] Final cost: {result.cost:.3f} | Time: {t:.8f}s | Path: {result.get_path()}")


def log_results(df, removed_percent, symmetrical, mode, size, fn_to_eval):
    gc.collect()
    ipython_memory_footprint = memory_usage(-1, max_usage=True, max_iterations=1)
    max_alloc, fnresult = memory_usage((fn_to_eval, (), {}), max_usage=True, retval=True, max_iterations=1)
    (result, t) = fnresult
    alloc_diff = max_alloc-ipython_memory_footprint
    df.append({'size': size, 'removed_percent': removed_percent, 
                'symmetrical' : symmetrical, 'alg_name': mode, 
                'cost': result.cost, 'time': t, 
                'path': result.get_path(), 'max_alloc': alloc_diff})
    print(f"    [{mode} | n={size}] Cost: {result.cost:.2f} | Time: {t:.4f}s | Path: {result.get_path()} | Max alloc {alloc_diff:.4f} MB")


In [5]:
from queue import PriorityQueue

def full_search(graph, start, mode='bfs'):
    start_time = time.time()

    pop_idx = {'bfs': 0, 'dfs': -1}
    candidate_nodes = []
    starting_point = State(start)
    candidate_nodes.append(starting_point)
    leaves = []
    while candidate_nodes:
        s = candidate_nodes.pop(pop_idx[mode])
        if is_goal_state(s, starting_point, graph):
            leaves.append(s)
        new_candidates = s.expand(graph)
        candidate_nodes.extend(new_candidates)
    final_best = min(leaves, key=lambda x: x.cost)

    total_time = time.time() - start_time
    return final_best, total_time


def nearest_neighbour(graph, start):
    start_time = time.time()
    starting_point = State(start)
    current_point = starting_point
    dead_ends = []
    while True:
        neighbours = current_point.expand(graph)
        neighbours = [n for n in neighbours if n not in dead_ends]  # exclude dead ends from calculation
        if neighbours:
            nn = min(neighbours, key=lambda x: x.cost)
            current_point = nn
            if is_goal_state(current_point, starting_point, graph):  # we are done
                break
        else:  # no neighbours
            if is_goal_state(current_point, starting_point, graph):  # leaf satisfies full path - we are done
                break
            else:  # we are stuck
                # print(f"stuck at {current_point}")
                if current_point not in dead_ends:
                    dead_ends.append(current_point)
                current_point = current_point.parent  # backtrack
    total_time = time.time() - start_time
    return current_point, total_time


def heuristic_search(graph, start, heuristic):
    start_time = time.time()
    candidate_nodes = PriorityQueue()
    starting_point = State(start)
    candidate_nodes.put((0, starting_point))

    while candidate_nodes:
        _, current_node = candidate_nodes.get()
        if is_goal_state(current_node, starting_point, graph):
            break
        new_candidates = current_node.expand(graph)
        if new_candidates:
            for candidate in new_candidates:
                predicted_cost = candidate.cost + heuristic(candidate, graph)
                candidate_nodes.put((predicted_cost, candidate))
    total_time = time.time() - start_time
    return current_node, total_time


def dijkstra_heuristic(current, graph):
    return 0


def a_star_min_heuristic(current, graph):
    # min cost of remaining paths * number of remaining cities to visit
    cities_to_visit = len(graph.vertices) - len(current.previous) + 1 # return path
    visited_so_far = current.visited_edges
    remaining_edges = graph.all_edges.copy()
    for visited_start, visited_end in visited_so_far:
        if (visited_start, visited_end) in remaining_edges:
            remaining_edges.remove((visited_start, visited_end))
    cost_list = list(map(lambda edge: graph.distances[edge[0]][edge[1]], remaining_edges))
    if len(remaining_edges) == 0:
        return 0
    return min(cost_list) * cities_to_visit


def a_star_avg_heuristic(current, graph):
    # average cost of remaining paths * number of remaining cities to visit
    cities_to_visit = len(graph.vertices) - len(current.previous) + 1 # return path
    visited_so_far = current.visited_edges
    remaining_edges = graph.all_edges.copy()
    for visited_start, visited_end in visited_so_far:
        if ((visited_start, visited_end) in remaining_edges):
            remaining_edges.remove((visited_start, visited_end))
    cost_list = list(map(lambda edge: graph.distances[edge[0]][edge[1]], remaining_edges))
    if len(remaining_edges) == 0:
        return 0
    return (sum(cost_list) / len(cost_list)) * cities_to_visit

def aco(graph, ants, iterations, evaporation=0.2, alpha=1, beta=2):
    start_time = time.time()
    #initialise pheromone matrix
    phero_matrix = {}
    for city in graph.vertices.keys():
        phero_matrix[city] = {}
        for other_city in graph.vertices.keys():
            phero_matrix[city][other_city] = 1
    #for each iteration
    for i in range(iterations):
        #initalize empty list of completed tours
        completed_tours = []
        for ant in range(ants):
            #pick random vertex to start from
            starting_point = State(random.choice([*graph.vertices.keys()]))
            current_node = starting_point
            while True:
                if(is_goal_state(current_node, starting_point, graph)):
                    completed_tours.append(current_node)
                    break
                children = current_node.expand(graph)
                if children:
                    nearby_cities = [point.pos for point in children]
                    #calculate total pheromones on nearby cities
                    nearby_phero = [(phero_matrix[current_node.pos][end_point]**alpha) * \
                                    (1/graph.distances[current_node.pos][end_point]**beta) \
                                    for end_point in nearby_cities] #pheromone^alpha * visibility^beta
                    total_nearby_phero = sum(nearby_phero)
                    next_node = np.random.choice(children, p=np.divide(nearby_phero, total_nearby_phero))
                    current_node = next_node
                else:
                    #the ant can't reach, we let it die
                    break
        #evaporate
        for phero_from in phero_matrix.keys():
            for phero_to in phero_matrix.keys():
                phero_matrix[phero_from][phero_to] *= 1 - evaporation
        #apply pheromones for each completed tour
        for completed_tour in completed_tours:
            for start, end in zip(completed_tour.previous, completed_tour.previous[1:]):
                phero_matrix[start][end] += 1/completed_tour.cost
    best = min(completed_tours, key=lambda x: x.cost)
                                                 
    return best, time.time() - start_time

In [6]:
import time
import pandas as pd

removed_percents = [0, 20]

df = []
for n in range(5, 21):
    for removed_percent in [0, 20]:
        for symmetrical in [True, False]:
            print(f"Trying n = {n} | removed {removed_percent} % | symmetrical - {symmetrical}")
            graph = Graph(n, removed_percent=removed_percent, symmetrical=symmetrical)
            inital_city = 'A'
            
            if(n < 10):
                log_results(df, removed_percent, symmetrical, "dfs        ", n, lambda : full_search(graph, inital_city, "dfs"))
                log_results(df, removed_percent, symmetrical, "bfs        ", n, lambda : full_search(graph, inital_city, "bfs"))
            log_results(df, removed_percent, symmetrical, "nn         ", n, lambda : nearest_neighbour(graph, inital_city))
            if(n < 11):
                log_results(df, removed_percent, symmetrical, "dijkstra   ", n, lambda : heuristic_search(graph, inital_city, dijkstra_heuristic))
            if(n < 13):
                log_results(df, removed_percent, symmetrical, "a_star_min ", n, lambda : heuristic_search(graph, inital_city, a_star_min_heuristic))
            log_results(df, removed_percent, symmetrical, "a_star_avg ", n, lambda : heuristic_search(graph, inital_city, a_star_avg_heuristic))
            
            log_results(df, removed_percent, symmetrical, "aco        ", n, lambda : aco(graph, 25, 100))
            gc.collect()
#df = pd.DataFrame(df, columns = ['size', 'removed_percent', 'symmetrical', 'alg_name', 'cost', 'time', 'path', 'max_alloc'])
#df.to_excel(f'lab_1_results.xlsx')

Trying n = 5 | removed 0 % | symmetrical - True
    [dfs         | n=5] Cost: 535.27 | Time: 0.0000s | Path: A>B>E>C>D>A | Max alloc 0.3203 MB
    [bfs         | n=5] Cost: 535.27 | Time: 0.0000s | Path: A>B>E>C>D>A | Max alloc 0.0000 MB
    [nn          | n=5] Cost: 555.68 | Time: 0.0000s | Path: A>D>C>E>B>A | Max alloc 0.0039 MB
    [dijkstra    | n=5] Cost: 535.27 | Time: 0.0010s | Path: A>B>E>C>D>A | Max alloc 0.0000 MB
    [a_star_min  | n=5] Cost: 535.27 | Time: 0.0010s | Path: A>B>E>C>D>A | Max alloc 0.0000 MB
    [a_star_avg  | n=5] Cost: 555.68 | Time: 0.0000s | Path: A>D>C>E>B>A | Max alloc 0.0000 MB
    [aco         | n=5] Cost: 535.27 | Time: 0.4274s | Path: C>D>A>B>E>C | Max alloc 0.1211 MB
Trying n = 5 | removed 0 % | symmetrical - False
    [dfs         | n=5] Cost: 430.75 | Time: 0.0010s | Path: A>D>B>C>E>A | Max alloc 0.0000 MB
    [bfs         | n=5] Cost: 430.75 | Time: 0.0000s | Path: A>D>B>C>E>A | Max alloc 0.0000 MB
    [nn          | n=5] Cost: 465.92 | Time: 0.0

    [nn          | n=7] Cost: 685.70 | Time: 0.0000s | Path: A>F>E>C>B>D>G>A | Max alloc 0.0000 MB
    [dijkstra    | n=7] Cost: 537.47 | Time: 0.0090s | Path: A>D>B>C>E>G>F>A | Max alloc 0.0000 MB
    [a_star_min  | n=7] Cost: 537.47 | Time: 0.0060s | Path: A>D>B>C>E>G>F>A | Max alloc 0.0000 MB
    [a_star_avg  | n=7] Cost: 653.72 | Time: 0.0000s | Path: A>F>E>C>D>B>G>A | Max alloc 0.0000 MB
    [aco         | n=7] Cost: 537.47 | Time: 0.6736s | Path: G>F>A>D>B>C>E>G | Max alloc 0.0000 MB
Trying n = 8 | removed 0 % | symmetrical - True
    [dfs         | n=8] Cost: 551.64 | Time: 0.2322s | Path: A>G>B>D>C>E>F>H>A | Max alloc 8.7422 MB
    [bfs         | n=8] Cost: 551.64 | Time: 0.2352s | Path: A>G>B>D>C>E>F>H>A | Max alloc 10.0195 MB
    [nn          | n=8] Cost: 640.63 | Time: 0.0010s | Path: A>F>H>E>C>D>B>G>A | Max alloc 0.0000 MB
    [dijkstra    | n=8] Cost: 551.64 | Time: 0.0450s | Path: A>G>B>D>C>E>F>H>A | Max alloc 0.6250 MB
    [a_star_min  | n=8] Cost: 551.64 | Time: 0.0951s

    [aco         | n=10] Cost: 516.85 | Time: 1.1270s | Path: G>J>B>E>I>C>A>F>D>H>G | Max alloc 0.0000 MB
Trying n = 10 | removed 20 % | symmetrical - False
    [nn          | n=10] Cost: 629.14 | Time: 0.0000s | Path: A>D>G>B>F>C>H>I>E>J>A | Max alloc 0.0000 MB
    [dijkstra    | n=10] Cost: 549.53 | Time: 3.2899s | Path: A>D>G>B>F>J>E>I>H>C>A | Max alloc 150.0195 MB
    [a_star_min  | n=10] Cost: 549.53 | Time: 3.3741s | Path: A>D>G>B>F>J>E>I>H>C>A | Max alloc 58.3750 MB
    [a_star_avg  | n=10] Cost: 622.06 | Time: 0.0010s | Path: A>D>B>F>G>C>H>I>E>J>A | Max alloc 0.0000 MB
    [aco         | n=10] Cost: 549.53 | Time: 1.1185s | Path: D>G>B>F>J>E>I>H>C>A>D | Max alloc 0.0000 MB
Trying n = 11 | removed 0 % | symmetrical - True
    [nn          | n=11] Cost: 738.66 | Time: 0.0010s | Path: A>H>I>K>G>F>C>E>D>B>J>A | Max alloc 0.0000 MB
    [a_star_min  | n=11] Cost: 656.25 | Time: 23.2404s | Path: A>H>I>K>J>G>B>D>E>C>F>A | Max alloc 417.5703 MB
    [a_star_avg  | n=11] Cost: 738.90 | Ti

    [a_star_avg  | n=15] Cost: 784.44 | Time: 0.0040s | Path: A>C>J>D>M>F>H>N>B>O>E>K>L>G>I>A | Max alloc 0.0000 MB
    [aco         | n=15] Cost: 772.10 | Time: 2.1183s | Path: E>K>H>F>M>D>L>G>I>C>J>A>N>O>B>E | Max alloc 0.0000 MB
Trying n = 15 | removed 20 % | symmetrical - True
    [nn          | n=15] Cost: 757.98 | Time: 0.0000s | Path: A>O>N>J>L>E>H>I>F>K>C>B>G>M>D>A | Max alloc 0.0000 MB
    [a_star_avg  | n=15] Cost: 757.98 | Time: 0.0050s | Path: A>O>N>J>L>E>H>I>F>K>C>B>G>M>D>A | Max alloc 0.0000 MB
    [aco         | n=15] Cost: 757.98 | Time: 2.0869s | Path: O>N>J>L>E>H>I>F>K>C>B>G>M>D>A>O | Max alloc 0.0000 MB
Trying n = 15 | removed 20 % | symmetrical - False
    [nn          | n=15] Cost: 752.35 | Time: 0.0010s | Path: A>M>G>E>D>B>F>O>C>N>J>K>I>L>H>A | Max alloc 0.0000 MB
    [a_star_avg  | n=15] Cost: 671.54 | Time: 0.0050s | Path: A>M>G>E>D>B>F>O>C>N>J>K>H>I>L>A | Max alloc 0.0000 MB
    [aco         | n=15] Cost: 650.50 | Time: 2.1009s | Path: K>J>N>I>O>L>B>E>D>M>A>G>H

    [a_star_avg  | n=20] Cost: 1009.05 | Time: 0.0160s | Path: A>J>F>S>G>L>R>N>I>M>K>E>D>Q>H>T>P>B>C>O>A | Max alloc 0.0000 MB
    [aco         | n=20] Cost: 863.32 | Time: 3.4464s | Path: Q>J>A>S>F>G>L>R>N>I>P>B>C>O>E>K>M>D>H>T>Q | Max alloc 0.0000 MB
Trying n = 20 | removed 20 % | symmetrical - True
    [nn          | n=20] Cost: 976.16 | Time: 0.0000s | Path: A>F>D>P>Q>J>B>H>E>S>L>N>I>T>M>K>C>G>R>O>A | Max alloc 0.0000 MB
    [a_star_avg  | n=20] Cost: 939.69 | Time: 0.0150s | Path: A>F>D>P>Q>J>B>H>E>S>L>N>I>T>M>K>C>R>G>O>A | Max alloc 0.0000 MB
    [aco         | n=20] Cost: 859.38 | Time: 3.3983s | Path: M>K>C>R>D>F>A>G>P>Q>J>O>H>B>E>S>I>N>L>T>M | Max alloc 0.0000 MB
Trying n = 20 | removed 20 % | symmetrical - False
    [nn          | n=20] Cost: 1007.62 | Time: 0.0000s | Path: A>L>J>K>D>C>E>M>I>N>Q>R>O>H>P>S>F>B>T>G>A | Max alloc 0.0000 MB
    [a_star_avg  | n=20] Cost: 1007.62 | Time: 0.0160s | Path: A>L>J>K>D>C>E>M>I>N>Q>R>O>H>P>S>F>B>T>G>A | Max alloc 0.0000 MB
    [aco      